In [22]:
import pandas as pd
import numpy as np
from dune_client.client import DuneClient
from flipside import Flipside
import plotly
import datetime as dt
from dotenv import load_dotenv
from prophet import Prophet
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


# import yfinance as yf

from utils import flipside_api_results
from sql import trader_classifier_query

In [23]:
load_dotenv()

True

In [24]:
dune_api_key = os.getenv('DUNE_API_KEY')
flipside_api_key = os.getenv('FLIPSIDE_API_KEY')
dune = DuneClient(dune_api_key)
print(dune_api_key, flipside_api_key)

CPh7RV9x58ODbTuhYEA9lnpD4NkIvyEi 4604d4d2-ccf3-4864-90b4-db6bf13c663b


# Data Retrieval 

## Dune Queries

Uniswap Arbitrum Pool Data
https://dune.com/queries/3930878

Arbitrum Gas Data
https://dune.com/queries/3931017

In [25]:
def dune_api_results(query_num, save_csv=False, csv_path=None):
    results = dune.get_latest_result(query_num)
    df = pd.DataFrame(results.result.rows)

    if save_csv and csv_path:
        df.to_csv(csv_path, index=False)
    return df


arb_pool_path = 'data/arb_pool_data.csv'
arb_pool_df = dune_api_results(3930878, True, arb_pool_path)
arb_pool_df

In [26]:
arb_pool_path = 'data/arb_pool_data.csv'
arb_pool_df = pd.read_csv(arb_pool_path)
arb_pool_df.columns = [f'arbitrum_{col}' if col != 'day' else col for col in arb_pool_df.columns]
print(arb_pool_df.columns)

Index(['arbitrum_avg_liquidity', 'day', 'arbitrum_fee_apr',
       'arbitrum_fee_tier', 'arbitrum_fees_usd', 'arbitrum_lp_addr',
       'arbitrum_num_trades', 'arbitrum_token_pair', 'arbitrum_tvl_usd',
       'arbitrum_volume_to_tvl', 'arbitrum_volume_usd'],
      dtype='object')


arbitrum_gas = dune.get_latest_result(3931017)
arbitrum_gas_df = pd.DataFrame(arbitrum_gas.result.rows)
arbitrum_gas_path = 'data/arb_gas.csv'
arbitrum_gas_df.to_csv(arbitrum_gas_path, index=False)

In [27]:
arbitrum_gas_path = 'data/arb_gas.csv'
arbitrum_gas_df = pd.read_csv(arbitrum_gas_path)
if 'Unnamed: 0' in arbitrum_gas_df.columns:
    arbitrum_gas_df = arbitrum_gas_df.drop('Unnamed: 0', axis=1)
arbitrum_gas_df.columns = [f'arbitrum_{col}' if col != 'dt' else col for col in arbitrum_gas_df.columns]
arbitrum_gas_df.columns


Index(['dt', 'arbitrum_gas_usd_per_tx', 'arbitrum_median_gas_usd'], dtype='object')

Uniswap Optimism Pool Data
https://dune.com/queries/3930989

Optimism Gas Data 
https://dune.com/queries/3930989

op_pool_path = 'data/op_pool_data.csv'
op_pool_df = dune_api_results(3930989, True, op_pool_path)
op_pool_df

In [28]:
op_pool_path = 'data/op_pool_data.csv'
op_pool_df = pd.read_csv(op_pool_path)
op_pool_df.columns = [f'optimism_{col}' if col != 'day' else col for col in op_pool_df.columns]
print(op_pool_df.columns)

Index(['optimism_avg_liquidity', 'day', 'optimism_fee_apr',
       'optimism_fee_tier', 'optimism_fees_usd', 'optimism_lp_addr',
       'optimism_num_trades', 'optimism_token_pair', 'optimism_tvl_usd',
       'optimism_volume_to_tvl', 'optimism_volume_usd'],
      dtype='object')


optimism_gas = dune.get_latest_result_dataframe(3931019)
optimism_gas_path = 'data/op_gas.csv'
optimism_gas.to_csv(optimism_gas_path, index=False)

In [29]:
optimism_gas_path = 'data/op_gas.csv'
optimism_gas_df = pd.read_csv(optimism_gas_path)
optimism_gas_df = optimism_gas_df.drop('_col3', axis=1) if '_col3' in optimism_gas_df.columns else optimism_gas_df
optimism_gas_df.columns = [f'optimism_{col}' if col != 'dt' else col for col in optimism_gas_df.columns]
optimism_gas_df.columns

Index(['dt', 'optimism_gas_usd_per_tx', 'optimism_median_gas_usd'], dtype='object')

Uniswap Base Pool Data
https://dune.com/queries/3930954

Base Gas Data
https://dune.com/queries/3931021

base_pool_path = 'data/base_pool_data.csv'
base_pool_df = dune_api_results(3930954, True, base_pool_path)
base_pool_df

In [30]:
base_pool_path = 'data/base_pool_data.csv'
base_pool_df = pd.read_csv(base_pool_path)
base_pool_df.columns = [f'base_{col}' if col != 'day' else col for col in base_pool_df.columns]
print(base_pool_df.columns)

Index(['base_avg_liquidity', 'day', 'base_fee_apr', 'base_fee_tier',
       'base_fees_usd', 'base_lp_addr', 'base_num_trades', 'base_token_pair',
       'base_tvl_usd', 'base_volume_to_tvl', 'base_volume_usd'],
      dtype='object')


base_gas = dune.get_latest_result_dataframe(3931021)
base_gas_path = 'data/base_gas.csv'
base_gas.to_csv(base_gas_path, index=False)

In [31]:
base_gas_path = 'data/base_gas.csv'
base_gas_df = pd.read_csv(base_gas_path)
base_gas_df.columns = [f'base_{col}' if col != 'dt' else col for col in base_gas_df.columns]
base_gas_df.columns

Index(['dt', 'base_gas_usd_per_tx', 'base_median_gas_usd'], dtype='object')

## Flipside Queries 

Trader Classifier - Arbitrum Data
https://flipsidecrypto.xyz/Brandyn/q/7NlPxrKU5KQb/2024-07-20-06-36-pm

classifier_data_path = 'data/classifier.csv'
trader_classifier_data = flipside_api_results(trader_classifier_query, flipside_api_key)
print(trader_classifier_data)
trader_classifier_data.to_csv(classifier_data_path, index=False)

In [32]:
classifier_data_path = 'data/classifier.csv'
trader_classifier_df = pd.read_csv(classifier_data_path)
trader_classifier_df.drop(columns=['__row_index'], inplace=True)
trader_classifier_df.head()

,trader_type,dt,tx_count,total_volume_usd,avg_order_size_usd,unique_contracts
0,Professional,2024-07-21T15:00:00.000Z,928,1047810.93,1183.967153,104
1,Retail,2024-07-21T15:00:00.000Z,176,91856.49,540.332294,60
2,Professional,2024-07-21T14:00:00.000Z,7552,12913378.38,1766.536030,252
3,Retail,2024-07-21T14:00:00.000Z,1790,294765.21,170.680492,196
4,Professional,2024-07-21T13:00:00.000Z,4472,5883981.27,1367.731583,211


# Data Cleaning/Processing

Each token pair has several fee tiers and some have differente lp addresses.  

Can aggregate metrics to token pair and/or seperate each fee-tier out, aggregate by addresses

# Data Analysis

## Arbitrum

### Summary Statistics

In [33]:
print(arb_pool_df.describe())

       arbitrum_avg_liquidity  arbitrum_fee_apr  arbitrum_fees_usd  \
count            3.468000e+03      3.468000e+03       3.468000e+03   
mean             1.782774e+24      1.458773e-02       2.966866e+02   
std              7.617222e+24      5.095527e-02       7.349982e+02   
min              9.140293e+07      3.241851e-13       6.002412e-15   
25%              6.812052e+15      8.222112e-04       2.920990e-01   
50%              1.903847e+18      4.227947e-03       1.321334e+01   
75%              2.049945e+22      1.363400e-02       2.495379e+02   
max              4.600563e+25      1.606219e+00       8.186259e+03   

       arbitrum_num_trades  arbitrum_tvl_usd  arbitrum_volume_to_tvl  \
count          3468.000000      3.468000e+03            3.468000e+03   
mean            190.908304      9.310402e+06            5.508794e-02   
std             338.143117      1.496615e+07            1.409350e-01   
min               1.000000      6.708087e+00            2.982674e-13   
25%      

In [34]:
print(arbitrum_gas_df.describe())

       arbitrum_gas_usd_per_tx  arbitrum_median_gas_usd
count              2162.000000              2162.000000
mean                  0.023634                 0.012860
std                   0.221630                 0.129315
min                   0.004397                 0.001491
25%                   0.006888                 0.004532
50%                   0.008472                 0.005564
75%                   0.010704                 0.007182
max                   6.944713                 4.181090


### Data Visualizations

In [35]:
arb_pool_df

,arbitrum_avg_liquidity,day,arbitrum_fee_apr,arbitrum_fee_tier,arbitrum_fees_usd,arbitrum_lp_addr,arbitrum_num_trades,arbitrum_token_pair,arbitrum_tvl_usd,arbitrum_volume_to_tvl,arbitrum_volume_usd
0,1.854139e+23,2024-07-14 00:00:00.000 UTC,3.781496e-02,0.30%,386.685410,0x4cef551255ec96d89fec975446301b5c4e164c59,119,WETH-ZRO,3.732390e+06,3.453421e-02,1.288951e+05
1,2.075720e+16,2024-07-14 00:00:00.000 UTC,1.591761e-02,0.01%,3.056607,0x6f38e884725a116c9c7fbf208e79fe8828a2595f,287,USDC-WETH,7.008980e+04,4.360988e-01,3.056607e+04
2,1.965666e+12,2024-07-14 00:00:00.000 UTC,1.469532e-02,0.05%,412.274720,0x0e4831319a50228b9e450861297ab92dee15b44f,466,USDC-WBTC,1.024001e+07,8.052231e-02,8.245494e+05
3,4.911041e+18,2024-07-14 00:00:00.000 UTC,1.290974e-02,0.05%,1705.827484,0xc6962004f452be9203591991d15f6b388e09e8d0,845,USDC-WETH,4.822924e+07,7.073831e-02,3.411655e+06
4,4.950377e+15,2024-07-14 00:00:00.000 UTC,8.265621e-03,0.01%,0.540105,0x42161084d0672e1d3f26a9b53e653be2084ff19c,136,USDT-WETH,2.385039e+04,2.264554e-01,5.401049e+03
...,...,...,...,...,...,...,...,...,...,...,...
3463,4.005206e+14,2024-07-21 01:00:00.000 UTC,5.688889e-05,0.01%,0.002823,0x42161084d0672e1d3f26a9b53e653be2084ff19c,10,USDT-WETH,1.811063e+04,1.558600e-03,2.822722e+01
3464,2.830767e+17,2024-07-21 01:00:00.000 UTC,1.923425e-05,0.05%,2.789811,0x2f5e87c9312fa29aed5c179e456625d79015299c,11,WBTC-WETH,5.294103e+07,1.053932e-04,5.579623e+03
3465,1.792792e+14,2024-07-21 01:00:00.000 UTC,1.801927e-05,0.01%,0.001416,0x03a3be7ab4aa263d42d63b6cc594f4fb3d3f3951,2,WBTC-WETH,2.867966e+04,4.936787e-04,1.415854e+01
3466,2.355089e+21,2024-07-21 01:00:00.000 UTC,3.497822e-06,0.01%,0.012592,0x7cf803e8d82a50504180f417b8bc7a493c0a0503,4,DAI-USDC,1.313937e+06,9.583075e-05,1.259156e+02


In [36]:
filtered_cols = ['day','arbitrum_avg_liquidity','arbitrum_fees_usd','arbitrum_tvl_usd','arbitrum_volume_usd','arbitrum_num_trades','arbitrum_volume_to_tvl','arbitrum_token_pair']

In [37]:
arb_pool_df['day'] = pd.to_datetime(arb_pool_df['day'])

In [170]:
def fee_tier_cleaning(df, network):
    fee_tiers = df[f'{network}_fee_tier'].unique()
    merged_df = None
    
    # Determine the complete date range
    min_date = df['day'].min()
    max_date = df['day'].max()
    complete_date_range = pd.date_range(start=min_date, end=max_date, freq='H')

    for fee_tier in fee_tiers:
        # Filter the DataFrame for the specific fee tier
        filtered_df = df[df[f'{network}_fee_tier'] == fee_tier].copy()
        
        # Rename the columns to include the fee tier
        fee_tier_suffix = fee_tier.replace('.', '').replace('%', '')
        filtered_df.columns = [f'{col}_{fee_tier_suffix}' if col not in ['day', f'{network}_gas_usd_per_tx', f'{network}_median_gas_usd'] else col for col in filtered_df.columns]
        
        # Set the 'day' column as the index
        filtered_df.set_index('day', inplace=True)
        
        # Reindex the DataFrame to the complete date range and fill NaN values with 0
        filtered_df = filtered_df.reindex(complete_date_range, fill_value=0).rename_axis('day').reset_index()
        
        # Calculate the net change in liquidity
        liquidity_col = f'{network}_avg_liquidity_{fee_tier_suffix}'
        net_liquidity_col = f'{network}_net_liquidity_{fee_tier_suffix}'
        filtered_df[net_liquidity_col] = filtered_df[liquidity_col] - filtered_df[liquidity_col].shift(1)
        filtered_df[net_liquidity_col].fillna(0, inplace=True)

        # Merge with the main DataFrame
        if merged_df is None:
            merged_df = filtered_df
        else:
            merged_df = pd.merge(merged_df, filtered_df, on='day', how='inner')

    arbitrum_gas_df.rename(columns={'dt':'day'}, inplace=True)
    arbitrum_gas_df['day'] = pd.to_datetime(arbitrum_gas_df['day'])
    merged_df = merged_df.merge(arbitrum_gas_df, how='left', on=['day'])
        
    return merged_df

In [123]:
# All types of fee tiers in dataset 

fee_tiers_unique = arb_pool_df['arbitrum_fee_tier'].unique()
token_pairs_unique = arb_pool_df['arbitrum_token_pair'].unique()

print(f'token pairs: {token_pairs_unique}')

token pairs: ['WETH-ZRO' 'USDC-WETH' 'USDC-WBTC' 'USDT-WETH' 'WBTC-WETH' 'ARB-WETH'
 'WETH-XAI' 'WETH-wstETH' 'DAI-USDC']


In [171]:
def token_pair_df_cleaned(df, token_pair, network):
    token_pair_df = df[df[f'{network}_token_pair']==f'{token_pair}']
    cleaned_token_pair_df = fee_tier_cleaning(token_pair_df, network)
    return cleaned_token_pair_df

In [172]:
usdc_weth = token_pair_df_cleaned(arb_pool_df, 'USDC-WETH', 'arbitrum') 
usdc_weth.columns

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_21460\3561558891.py:8: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_21460\3561558891.py:28: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_21460\3561558891.py:28: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas

Index(['day', 'arbitrum_avg_liquidity_001', 'arbitrum_fee_apr_001',
       'arbitrum_fee_tier_001', 'arbitrum_fees_usd_001',
       'arbitrum_lp_addr_001', 'arbitrum_num_trades_001',
       'arbitrum_token_pair_001', 'arbitrum_tvl_usd_001',
       'arbitrum_volume_to_tvl_001', 'arbitrum_volume_usd_001',
       'arbitrum_net_liquidity_001', 'arbitrum_avg_liquidity_005',
       'arbitrum_fee_apr_005', 'arbitrum_fee_tier_005',
       'arbitrum_fees_usd_005', 'arbitrum_lp_addr_005',
       'arbitrum_num_trades_005', 'arbitrum_token_pair_005',
       'arbitrum_tvl_usd_005', 'arbitrum_volume_to_tvl_005',
       'arbitrum_volume_usd_005', 'arbitrum_net_liquidity_005',
       'arbitrum_avg_liquidity_030', 'arbitrum_fee_apr_030',
       'arbitrum_fee_tier_030', 'arbitrum_fees_usd_030',
       'arbitrum_lp_addr_030', 'arbitrum_num_trades_030',
       'arbitrum_token_pair_030', 'arbitrum_tvl_usd_030',
       'arbitrum_volume_to_tvl_030', 'arbitrum_volume_usd_030',
       'arbitrum_net_liquidity_0

In [40]:
arb_pool_df[(arb_pool_df['arbitrum_token_pair']=='DAI-USDC')]['arbitrum_fee_tier'].unique()

array(['0.01%', '0.05%', '0.30%'], dtype=object)

In [41]:
weth_zro_df = arb_pool_df[(arb_pool_df['arbitrum_token_pair']=='WETH-ZRO')]
weth_zro = fee_tier_cleaning(weth_zro_df) 

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_21460\1703551462.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  complete_date_range = pd.date_range(start=min_date, end=max_date, freq='H')
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_21460\1703551462.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df[net_liquidity_col].fillna(0, inplace=True)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_21460\1703551462.py:28: FutureWarning: A value is trying t

In [42]:
weth_zro.columns

Index(['day', 'arbitrum_avg_liquidity_030', 'arbitrum_fee_apr_030',
       'arbitrum_fee_tier_030', 'arbitrum_fees_usd_030',
       'arbitrum_lp_addr_030', 'arbitrum_num_trades_030',
       'arbitrum_token_pair_030', 'arbitrum_tvl_usd_030',
       'arbitrum_volume_to_tvl_030', 'arbitrum_volume_usd_030',
       'arbitrum_net_liquidity_030', 'arbitrum_avg_liquidity_100',
       'arbitrum_fee_apr_100', 'arbitrum_fee_tier_100',
       'arbitrum_fees_usd_100', 'arbitrum_lp_addr_100',
       'arbitrum_num_trades_100', 'arbitrum_token_pair_100',
       'arbitrum_tvl_usd_100', 'arbitrum_volume_to_tvl_100',
       'arbitrum_volume_usd_100', 'arbitrum_net_liquidity_100',
       'arbitrum_gas_usd_per_tx', 'arbitrum_median_gas_usd'],
      dtype='object')

In [43]:
dai_usdc_df = arb_pool_df[(arb_pool_df['arbitrum_token_pair']=='DAI-USDC')]
dai_usdc = fee_tier_cleaning(dai_usdc_df) 


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_21460\1703551462.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  complete_date_range = pd.date_range(start=min_date, end=max_date, freq='H')
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_21460\1703551462.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df[net_liquidity_col].fillna(0, inplace=True)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_21460\1703551462.py:28: FutureWarning: A value is trying t

In [44]:
dai_usdc.columns

Index(['day', 'arbitrum_avg_liquidity_001', 'arbitrum_fee_apr_001',
       'arbitrum_fee_tier_001', 'arbitrum_fees_usd_001',
       'arbitrum_lp_addr_001', 'arbitrum_num_trades_001',
       'arbitrum_token_pair_001', 'arbitrum_tvl_usd_001',
       'arbitrum_volume_to_tvl_001', 'arbitrum_volume_usd_001',
       'arbitrum_net_liquidity_001', 'arbitrum_avg_liquidity_005',
       'arbitrum_fee_apr_005', 'arbitrum_fee_tier_005',
       'arbitrum_fees_usd_005', 'arbitrum_lp_addr_005',
       'arbitrum_num_trades_005', 'arbitrum_token_pair_005',
       'arbitrum_tvl_usd_005', 'arbitrum_volume_to_tvl_005',
       'arbitrum_volume_usd_005', 'arbitrum_net_liquidity_005',
       'arbitrum_avg_liquidity_030', 'arbitrum_fee_apr_030',
       'arbitrum_fee_tier_030', 'arbitrum_fees_usd_030',
       'arbitrum_lp_addr_030', 'arbitrum_num_trades_030',
       'arbitrum_token_pair_030', 'arbitrum_tvl_usd_030',
       'arbitrum_volume_to_tvl_030', 'arbitrum_volume_usd_030',
       'arbitrum_net_liquidity_0

weth_zro_30 = arb_pool_df[(arb_pool_df['arbitrum_token_pair']=='WETH-ZRO') & (arb_pool_df['arbitrum_fee_tier']=='0.30%')]
# weth_zro = weth_zro[filtered_cols]
weth_zro_30.columns = [f'{col}_.30%' if col != 'day' else col for col in weth_zro_30.columns]
weth_zro_30.columns

# arb_pool_df[(arb_pool_df['arbitrum_token_pair']=='WETH-ZRO')]['arbitrum_fee_tier'].unique()

weth_zro_100 = arb_pool_df[(arb_pool_df['arbitrum_token_pair']=='WETH-ZRO') & (arb_pool_df['arbitrum_fee_tier']=='1.00%')]
weth_zro_100.columns = [f'{col}_1.0%' if col != 'day' else col for col in weth_zro_100.columns]
weth_zro_100.columns


weth_zro_30['arbitrum_net_liquidity_.30%'] = weth_zro_30['arbitrum_avg_liquidity_.30%'] - weth_zro_30['arbitrum_avg_liquidity_.30%'].shift(1)
weth_zro_30['arbitrum_net_liquidity_.30%'].fillna(0, inplace=True) 
print(weth_zro_30[['day', 'arbitrum_avg_liquidity_.30%', 'arbitrum_net_liquidity_.30%']].head())

weth_zro_100['arbitrum_net_liquidity_1.0%'] = weth_zro_100['arbitrum_avg_liquidity_1.0%'] - weth_zro_100['arbitrum_avg_liquidity_1.0%'].shift(1)
weth_zro_100['arbitrum_net_liquidity_1.0%'].fillna(0, inplace=True) 
print(weth_zro_100[['day', 'arbitrum_avg_liquidity_1.0%', 'arbitrum_net_liquidity_1.0%']].head())

weth_zro = pd.merge(weth_zro_30, weth_zro_100, how='left', on=['day']) 
weth_zro = weth_zro.fillna(0)

In [143]:
def corr_matrix_graph(df, fee_tier):
    fee_tier_cols = [f'arbitrum_avg_liquidity_{fee_tier}', f'arbitrum_fees_usd_{fee_tier}', f'arbitrum_tvl_usd_{fee_tier}', f'arbitrum_volume_usd_{fee_tier}']
    corr_matrix = df[fee_tier_cols].corr()
    fig = go.Figure(data=go.Heatmap(
                   z=corr_matrix.values,
                   x=corr_matrix.columns,
                   y=corr_matrix.columns,
                   colorscale='Viridis',
                   zmin=-1, zmax=1))

    fig.update_layout(
        title='Correlation Matrix',
        xaxis_nticks=36
    )

    fig.show()
    return fig 

In [144]:
def trades_to_gas_chart(df, barmode='group'):
    df_fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    suffixes = ['_030', '_005', '_001', '_100']
    for suffix in suffixes:
        liq_column = f'arbitrum_num_trades{suffix}'
        if liq_column in df.columns:
            df_fig.add_trace(
                go.Bar(
                    x=df['day'],
                    y=df[liq_column],
                    name=f'{suffix.replace("_", "")}% # of Trades'
                ),
                secondary_y=False
            )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_median_gas_usd'],
            name='Median Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_gas_usd_per_tx'],
            name='Avg Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.update_layout(
        title='Trades to Gas Comparison',
        barmode=barmode  # Set the bar mode to either 'group' for side-by-side or 'stack' for stacked
    )

    df_fig.update_xaxes(title_text="Date")

    df_fig.show()

    return df_fig

In [145]:
def liq_to_gas_chart(df, barmode='group'):
    df_fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    suffixes = ['_030', '_005', '_001', '_100']
    for suffix in suffixes:
        liq_column = f'arbitrum_avg_liquidity{suffix}'
        if liq_column in df.columns:
            df_fig.add_trace(
                go.Bar(
                    x=df['day'],
                    y=df[liq_column],
                    name=f'{suffix.replace("_", "")}% Liquidity'
                ),
                secondary_y=False
            )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_median_gas_usd'],
            name='Median Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_gas_usd_per_tx'],
            name='Avg Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.update_layout(
        title='Liquidity to Gas Comparison',
        barmode=barmode  # Set the bar mode to either 'group' for side-by-side or 'stack' for stacked
    )

    df_fig.update_xaxes(title_text="Date")

    df_fig.show()

    return df_fig

In [146]:
def net_liq_to_gas_chart(df, barmode='group'):
    df_fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    suffixes = ['_030', '_005', '_001', '_100']
    for suffix in suffixes:
        net_liq_column = f'arbitrum_net_liquidity{suffix}'
        if net_liq_column in df.columns:
            df_fig.add_trace(
                go.Bar(
                    x=df['day'],
                    y=df[net_liq_column],
                    name=f'{suffix.replace("_", "")}% Net Liquidity Added'
                ),
                secondary_y=False
            )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_median_gas_usd'],
            name='Median Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_gas_usd_per_tx'],
            name='Avg Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.update_layout(
        title='Net Liquidity to Gas Comparison',
        barmode=barmode  # Set the bar mode to either 'group' for side-by-side or 'stack' for stacked
    )

    df_fig.update_xaxes(title_text="Date")

    df_fig.show()

    return df_fig

In [147]:
def vol_tvl_to_gas_chart(df, barmode='group'):
    df_fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    suffixes = ['_030', '_005', '_001', '_100']
    for suffix in suffixes:
        vol_tvl_column = f'arbitrum_volume_to_tvl{suffix}'
        if vol_tvl_column in df.columns:
            df_fig.add_trace(
                go.Bar(
                    x=df['day'],
                    y=df[vol_tvl_column],
                    name=f'{suffix.replace("_", "")}% Volume to TVL Ratio'
                ),
                secondary_y=False
            )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_median_gas_usd'],
            name='Median Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_gas_usd_per_tx'],
            name='Avg Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.update_layout(
        title='Volume/TVL Ratio to Gas Comparison',
        barmode=barmode  # Set the bar mode to either 'group' for side-by-side or 'stack' for stacked
    )

    df_fig.update_xaxes(title_text="Date")

    df_fig.show()

    return df_fig

In [148]:
def liquidity_to_trades_chart(df, barmode='group'):
    df_fig = make_subplots(specs=[[{"secondary_y": True}]])

    suffixes = ['_030', '_005', '_001', '_100']
    for suffix in suffixes:
        liquidity_column = f'arbitrum_avg_liquidity{suffix}'
        if liquidity_column in df.columns:
            df_fig.add_trace(
                go.Bar(
                    x=df['day'],
                    y=df[liquidity_column],
                    name=f'{suffix.replace("_", "")}% Liquidity'
                ),
                secondary_y=False
            )
            
    for suffix in suffixes:
        trades_column = f'arbitrum_num_trades{suffix}'
        if trades_column in df.columns:
            df_fig.add_trace(
                go.Scatter(
                x=df['day'],
                y=df[trades_column],
                name=f'{suffix.replace("_", "")}% # Of Trades',
                mode='lines'
            ),
            secondary_y=True
            )
    df_fig.update_layout(title='Liquidity to # of Trades Comparison',
        barmode=barmode)

    df_fig.update_xaxes(title_text="Date")

    df_fig.show()
    return df_fig


In [149]:
def gas_to_rev_chart(df, barmode='group'):
    df_fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    suffixes = ['_030', '_005', '_001', '_100']
    for suffix in suffixes:
        fee_column = f'arbitrum_fees_usd{suffix}'
        if fee_column in df.columns:
            df_fig.add_trace(
                go.Bar(
                    x=df['day'],
                    y=df[fee_column],
                    name=f'{suffix.replace("_", "")}% Fee Revenue'
                ),
                secondary_y=False
            )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_median_gas_usd'],
            name='Median Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.add_trace(
        go.Scatter(
            x=df['day'],
            y=df['arbitrum_gas_usd_per_tx'],
            name='Avg Gas',
            mode='lines'
        ),
        secondary_y=True
    )

    df_fig.update_layout(
        title='Fee Revenue to Gas Comparison',
        barmode=barmode  # Set the bar mode to either 'group' for side-by-side or 'stack' for stacked
    )

    df_fig.update_xaxes(title_text="Date")

    df_fig.show()

    return df_fig

### Volatile-Volatile Pair Visualizations
- WETH-ZRO

In [150]:
print([col for col in weth_zro.columns if col.startswith('arbitrum_avg_liquidity')])

['arbitrum_avg_liquidity_030', 'arbitrum_avg_liquidity_100']


In [151]:
weth_zro_corr_matrix = corr_matrix_graph(weth_zro, '030')

In [152]:
weth_zro_gas_to_rev_fig = gas_to_rev_chart(weth_zro, barmode='group')

In [153]:
weth_zro_liquidity_to_trades_fig = liquidity_to_trades_chart(weth_zro)

In [154]:
weth_zro_vol_tvl_ratio_fig = vol_tvl_to_gas_chart(weth_zro)

In [155]:
weth_zro_net_liq_to_gas_fig = net_liq_to_gas_chart(weth_zro)

In [156]:
weth_zro_liq_to_gas_fig = liq_to_gas_chart(weth_zro)

In [157]:
weth_zro_trades_to_gas_fig = trades_to_gas_chart(weth_zro)

### Volatile-Stable Pair Visualizations

- USDC-WETH

In [158]:
usdc_weth_gas_to_rev_fig = gas_to_rev_chart(usdc_weth, barmode='stack')

In [159]:
usdc_weth_liquidity_to_trades_fig = liquidity_to_trades_chart(usdc_weth, barmode='stack')

In [160]:
usdc_weth_vol_tvl_ratio_fig = vol_tvl_to_gas_chart(usdc_weth, barmode='stack')

In [161]:
usdc_weth_net_liq_to_gas_fig = net_liq_to_gas_chart(usdc_weth, barmode='stack')

In [162]:
usdc_weth_liq_to_gas_fig = liq_to_gas_chart(usdc_weth, barmode='stack')

In [163]:
usdc_weth_trades_to_gas_fig = trades_to_gas_chart(usdc_weth, barmode='stack')

### Stable-Stable Pair Visualizations
- DAI-USDC

In [164]:
dai_usdc_gas_to_rev_fig = gas_to_rev_chart(dai_usdc, barmode='group')

In [165]:
dai_usdc_liquidity_to_trades_fig = liquidity_to_trades_chart(dai_usdc, barmode='stack')

In [166]:
dai_usdc_vol_tvl_ratio_fig = vol_tvl_to_gas_chart(dai_usdc, barmode='stack')

In [167]:
dai_usdc_net_liq_to_gas_fig = net_liq_to_gas_chart(dai_usdc, barmode='stack')

In [168]:
dai_usdc_liq_to_gas_fig = liq_to_gas_chart(dai_usdc)

In [169]:
dai_usdc_trades_to_gas_fig = trades_to_gas_chart(dai_usdc)

## Optimism

## Base

## Trader Behavior

# Correlation Analysis